In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit, min, max, mean

In [2]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

24/04/25 09:04:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
districtes = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:trusted_zone/freshdata_trusted.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("query", '''
        SELECT DISTINCT b.District 
        FROM rent_price b
    ''') \
    .load()

districtes = [i[0] for i in districtes.collect()]

In [4]:
districtes

['Eixample',
 'Gracia',
 'Horta-Guinardo',
 'Nou_Barris',
 'Sants-Montjuic',
 'Sarria-Sant_Gervasi',
 'Sant_Marti',
 'Sant_Andreu',
 'Ciutat_Vella',
 'Les_Corts']

In [ ]:
# Dividir datos en entrenamiento y prueba
train_size = int(len(data) * 0.9)  # Usaremos el 70% de los datos como entrenamiento
train, test = data[:train_size], data[train_size:]

# Configurar y ajustar el modelo ARIMA automáticamente
model = auto_arima(train, start_p=0, start_q=0, max_p=3, max_q=3, seasonal=False, trace=True)
model.fit(train)

# Hacer predicciones
predictions = model.predict(n_periods=len(test))

# Calcular el error
error = np.sqrt(mean_squared_error(test, predictions))

print(f'Predicción del siguiente valor: {model.predict(n_periods=1)[0]:.2f}')
print(f'Error RMSE en conjunto de prueba: {error:.2f}')

# Visualizar resultados
plt.figure(figsize=(10, 6))
plt.plot(data, label='Serie Original')
plt.plot(np.arange(train_size, len(data)), predictions, label='Predicciones', linestyle='--')
plt.title(f'AutoARIMA Prediction (RMSE={error:.2f})')
plt.xlabel('Índice de Tiempo')
plt.ylabel('Valor')
plt.legend()
plt.show()
